# Do emissions-based temperature attribution using fair

- Calibration set v1.5.0 (historical observed to 2023)
- Emissions scenario CMIP7 preliminary historical extended with re-harmonized SSP2-4.5 (to 2024)

In [ ]:
import os

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import scipy.stats
import xarray as xr

In [ ]:
pl.style.use('../defaults.mplstyle')

In [ ]:
scenarios = ['ssp245', 'ssp245-noCH4', 'ssp245-noSO2']

In [ ]:
f = FAIR()
f.define_time(1750, 2025, 1)
f.define_scenarios(scenarios)
species, properties = read_properties('../data/calibration/species_configs_properties_1.5.0.csv')
f.define_species(species, properties)
f.ch4_method='Thornhill2021'
df_configs = pd.read_csv('../data/calibration/calibrated_constrained_parameters_1.5.0.csv', index_col=0)
f.define_configs(df_configs.index)
f.allocate()

In [ ]:
f.fill_from_csv(
    emissions_file='../data/emissions/ssp245_2022_harmon_1750-2024.csv',
    forcing_file='../data/forcing/volcanic_solar.csv',
)

In [ ]:
fill(
    f.forcing,
    f.forcing.sel(specie="Volcanic") * df_configs["forcing_scale[Volcanic]"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f.forcing,
    f.forcing.sel(specie="Solar") * df_configs["forcing_scale[Solar]"].values.squeeze(),
    specie="Solar",
)

f.fill_species_configs("../data/calibration/species_configs_properties_1.5.0.csv")
f.override_defaults("../data/calibration/calibrated_constrained_parameters_1.5.0.csv")

# initial conditions
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

In [ ]:
f.run()

In [ ]:
pl.plot(
    f.timebounds,
    (
        (f.temperature.sel(layer=0, scenario='ssp245')) - 
        (f.temperature.sel(layer=0, scenario='ssp245-noCH4'))
    ).median(dim='config')
)
pl.plot(
    f.timebounds,
    (
        (f.temperature.sel(layer=0, scenario='ssp245')) - 
        (f.temperature.sel(layer=0, scenario='ssp245-noSO2'))
    ).median(dim='config')
)

In [ ]:
f_noco2 = FAIR()
f_noco2.define_time(1750, 2025, 1)
f_noco2.define_scenarios(['ssp245', 'ssp245-noCO2', 'ssp245-noGHG'])
species, properties = read_properties('../data/calibration/species_configs_properties_1.5.0_noCO2.csv')
f_noco2.define_species(species, properties)
f_noco2.ch4_method='Thornhill2021'
df_configs = pd.read_csv('../data/calibration/calibrated_constrained_parameters_1.5.0.csv', index_col=0)
f_noco2.define_configs(df_configs.index)
f_noco2.allocate()

f_noco2.fill_from_csv(
    emissions_file='../data/emissions/ssp245_2022_harmon_1750-2024.csv',
    forcing_file='../data/forcing/external_forcing_noCO2.csv',
)

fill(
    f_noco2.forcing,
    f_noco2.forcing.sel(specie="Volcanic") * df_configs["forcing_scale[Volcanic]"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f_noco2.forcing,
    f_noco2.forcing.sel(specie="Solar") * df_configs["forcing_scale[Solar]"].values.squeeze(),
    specie="Solar",
)
fill(
    f_noco2.forcing,
    f_noco2.forcing.sel(specie="Land use") * df_configs["forcing_scale[Land use]"].values.squeeze(),
    specie="Land use",
)

f_noco2.fill_species_configs("../data/calibration/species_configs_properties_1.5.0_noCO2.csv")
f_noco2.override_defaults("../data/calibration/calibrated_constrained_parameters_1.5.0.csv")

# initial conditions
initialise(f_noco2.concentration, f_noco2.species_configs["baseline_concentration"])
initialise(f_noco2.forcing, 0)
initialise(f_noco2.temperature, 0)
initialise(f_noco2.cumulative_emissions, 0)
initialise(f_noco2.airborne_emissions, 0)

f_noco2.run()

In [ ]:
pl.plot(
    f_noco2.timebounds,
    (
        (f_noco2.temperature.sel(layer=0, scenario='ssp245')) - 
        (f_noco2.temperature.sel(layer=0, scenario='ssp245-noCO2'))
    ).median(dim='config')
)

In [ ]:
pl.plot(
    f_noco2.timebounds,
    (
        (f_noco2.temperature.sel(layer=0, scenario='ssp245')) - 
        (f_noco2.temperature.sel(layer=0, scenario='ssp245-noGHG'))
    ).median(dim='config')
)

In [ ]:
pl.plot(
    f.timebounds,
    (
        (f.forcing_sum.sel(scenario='ssp245')) - 
        (f.forcing_sum.sel(scenario='ssp245-noSO2'))
    ).median(dim='config')/1.31
)
pl.plot(
    f.timebounds,
    (
        (f.temperature.sel(layer=0, scenario='ssp245')) - 
        (f.temperature.sel(layer=0, scenario='ssp245-noSO2'))
    ).median(dim='config')
)

In [ ]:
pl.plot(
    (
        (f.forcing_sum.sel(scenario='ssp245')) - 
        (f.forcing_sum.sel(scenario='ssp245-noSO2'))
    ).median(dim='config')/1.31,
    (
        (f.temperature.sel(layer=0, scenario='ssp245')) - 
        (f.temperature.sel(layer=0, scenario='ssp245-noSO2'))
    ).median(dim='config')
)

In [ ]:
pl.plot(
    f.timebounds,
    (
        (f.forcing_sum.sel(scenario='ssp245')) - 
        (f.forcing_sum.sel(scenario='ssp245-noSO2'))
    ).median(dim='config')/(
        (f.temperature.sel(layer=0, scenario='ssp245')) - 
        (f.temperature.sel(layer=0, scenario='ssp245-noSO2'))
    ).median(dim='config')
)
pl.xlim(1850, 2025)
pl.ylim(0, 3)

In [ ]:
pl.plot(
    f.timebounds,
    (
        (f.forcing_sum.sel(scenario='ssp245')) - 
        (f.forcing_sum.sel(scenario='ssp245-noCH4'))
    ).median(dim='config')/1.31
)
pl.plot(
    f.timebounds,
    (
        (f.temperature.sel(layer=0, scenario='ssp245')) - 
        (f.temperature.sel(layer=0, scenario='ssp245-noCH4'))
    ).median(dim='config')
)

In [ ]:
pl.plot(
    f.timebounds,
    (
        (f.forcing_sum.sel(scenario='ssp245')) - 
        (f.forcing_sum.sel(scenario='ssp245-noCH4'))
    ).median(dim='config')
)

In [ ]:
pl.plot(
    (
        (f.forcing_sum.sel(scenario='ssp245')) - 
        (f.forcing_sum.sel(scenario='ssp245-noCH4'))
    ).median(dim='config')/1.31,
    (
        (f.temperature.sel(layer=0, scenario='ssp245')) - 
        (f.temperature.sel(layer=0, scenario='ssp245-noCH4'))
    ).median(dim='config')
)

In [ ]:
pl.plot(
    f.timebounds,
    (
        (f.forcing_sum.sel(scenario='ssp245')) - 
        (f.forcing_sum.sel(scenario='ssp245-noCH4'))
    ).median(dim='config')/(
        (f.temperature.sel(layer=0, scenario='ssp245')) - 
        (f.temperature.sel(layer=0, scenario='ssp245-noCH4'))
    ).median(dim='config')
)

pl.xlim(1850, 2025)
pl.ylim(0,3)

In [ ]:
pl.plot(
    f_noco2.timebounds,
    (
        (f_noco2.forcing_sum.sel(scenario='ssp245')) - 
        (f_noco2.forcing_sum.sel(scenario='ssp245-noCO2'))
    ).median(dim='config')/1.31
)
pl.plot(
    f_noco2.timebounds,
    (
        (f_noco2.temperature.sel(layer=0, scenario='ssp245')) - 
        (f_noco2.temperature.sel(layer=0, scenario='ssp245-noCO2'))
    ).median(dim='config')
)

In [ ]:
pl.plot(
    (
        (f_noco2.forcing_sum.sel(scenario='ssp245')) - 
        (f_noco2.forcing_sum.sel(scenario='ssp245-noCO2'))
    ).median(dim='config')/1.31,
    (
        (f_noco2.temperature.sel(layer=0, scenario='ssp245')) - 
        (f_noco2.temperature.sel(layer=0, scenario='ssp245-noCO2'))
    ).median(dim='config')
)

In [ ]:
pl.plot(
    f.timebounds,
    (
        (f_noco2.forcing_sum.sel(scenario='ssp245')) - 
        (f_noco2.forcing_sum.sel(scenario='ssp245-noCO2'))
    ).median(dim='config')/(
        (f_noco2.temperature.sel(layer=0, scenario='ssp245')) - 
        (f_noco2.temperature.sel(layer=0, scenario='ssp245-noCO2'))
    ).median(dim='config')
)

In [ ]:
data = np.array(
    [
        (
            (f.temperature.sel(layer=0, scenario='ssp245')) - 
            (f.temperature.sel(layer=0, scenario='ssp245-noCH4'))
        ).quantile(0.05, dim='config'),
        (
            (f.temperature.sel(layer=0, scenario='ssp245')) - 
            (f.temperature.sel(layer=0, scenario='ssp245-noCH4'))
        ).median(dim='config'),
        (
            (f.temperature.sel(layer=0, scenario='ssp245')) - 
            (f.temperature.sel(layer=0, scenario='ssp245-noCH4'))
        ).quantile(0.95, dim='config')
    ]
)
data = (data[:, :-1] + data[:, 1:])/2
data

In [ ]:
df_ch4 = pd.DataFrame(data.T, columns=['p5', 'median', 'p95'], index=f.timepoints)
os.makedirs('../output', exist_ok=True)
df_ch4.to_csv('../output/ch4_emissions_attribution.csv')

In [ ]:
pl.plot(f.concentration.sel(scenario='ssp245-noCH4', specie='CH4').median(dim='config'))

In [ ]:
pl.plot(f.concentration.sel(scenario='ssp245', specie='CH4').median(dim='config'))